In [1]:
from flask import *
import cv2
from get_sudoku_grid import *
from solve_sudoku import *
from imutils.video import VideoStream
import time
import threading

In [8]:
app = Flask(__name__)

vs = VideoStream(src=0).start()
#vs = cv2.VideoCapture(0)
#print(vs.isOpened())
time.sleep(2.0)
outputFrame = None

@app.route('/')
def index():
    return(render_template('index.html'))


def get_solved_sudoku():
    
    global vs, outputFrame
    val_init = {}
    frame_num = 0
    while True:
        frame = vs.read()
        img_r = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        processed_img = preprocess_image(img_r)
        req_points = get_corners(processed_img)
        warped_image = get_warped(img_r, req_points)
        squares = infer_grid(warped_image)

        try:
            sud_digits = get_sudoku(img = frame)
            sum_digits = sum([np.sum(i) for i in sud_digits])
            if (sum_digits)>0:
                digits = [i[np.newaxis,:,:,np.newaxis]/255 for i in sud_digits]
                grid_preds = [str(model_ch4.predict(i).argmax(-1)[0]+1)if np.sum(i)>0 else '.' for i in digits]
                valid_sum = sum([False if i in '0.' else True for i in grid_preds])
                if valid_sum>15:
                    val = solve(grid_preds)
                    if val and val != val_init:
                        val_init = val
                        empty = '0.'
                        fin_img = warped_image.copy()
                        for ind, i in enumerate(grid_preds):
                            if i in empty: 
                                x1 = (squares[ind][0][0]+squares[ind][1][0])/2
                                x2 = (squares[ind][0][1]+squares[ind][1][1])/2
                                fin_img = put_text(fin_img, val[list(val)[ind]], 
                                             ((x1+squares[ind][0][0])/2, (x2+squares[ind][1][1])/2))
                        outputFrame = fin_img.copy()
        except:
            continue
    

def generate():
    global outputFrame
    while True:
        if outputFrame is None: continue
        (flag, encodedImage) = cv2.imencode(".jpg", outputFrame)
        if not flag: continue
        yield(b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + bytearray(encodedImage) + b'\r\n')


@app.route('/video_feed')
def video_feed():
    return Response(generate(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    # defining server ip address and port
    t = threading.Thread(target = get_solved_sudoku)
    t.daemon = True
    t.start()
    print('started')
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app, use_debugger=True, threaded=True, use_reloader = False)

vs.stop()

 * Running on http://localhost:9000/ (Press CTRL+C to quit)


[[[250 252 252]
  [250 252 252]
  [251 252 252]
  ...
  [192 207 202]
  [193 208 201]
  [195 210 202]]

 [[250 252 252]
  [250 252 252]
  [250 252 252]
  ...
  [196 209 204]
  [194 208 200]
  [197 211 202]]

 [[250 252 252]
  [250 252 252]
  [249 253 251]
  ...
  [199 208 202]
  [197 208 200]
  [200 211 201]]

 ...

 [[255 238 211]
  [255 238 211]
  [255 238 211]
  ...
  [249 255 232]
  [249 255 232]
  [249 255 233]]

 [[255 235 211]
  [255 236 212]
  [255 236 212]
  ...
  [250 255 232]
  [250 255 232]
  [250 255 233]]

 [[254 233 211]
  [254 233 211]
  [255 234 212]
  ...
  [251 255 231]
  [251 255 232]
  [251 255 232]]]
started
[[[250 253 251]
  [250 253 251]
  [250 253 251]
  ...
  [197 207 198]
  [202 211 201]
  [202 212 200]]

 [[251 253 251]
  [251 253 251]
  [250 253 251]
  ...
  [202 211 202]
  [200 209 199]
  [200 210 198]]

 [[252 253 251]
  [251 253 251]
  [251 253 251]
  ...
  [202 209 201]
  [198 206 196]
  [201 209 198]]

 ...

 [[253 235 211]
  [255 236 212]
  [255 235 2

[[[247 253 252]
  [247 253 252]
  [248 253 252]
  ...
  [199 203 198]
  [206 211 203]
  [203 209 199]]

 [[246 253 252]
  [247 253 252]
  [248 253 252]
  ...
  [201 204 198]
  [205 209 201]
  [204 208 199]]

 [[246 253 252]
  [247 253 252]
  [248 253 252]
  ...
  [203 205 200]
  [203 206 200]
  [204 208 200]]

 ...

 [[255 234 212]
  [255 235 214]
  [255 236 214]
  ...
  [251 255 236]
  [251 255 237]
  [251 254 238]]

 [[255 233 214]
  [255 234 215]
  [255 234 215]
  ...
  [251 255 235]
  [251 255 236]
  [251 255 236]]

 [[255 232 215]
  [255 233 216]
  [255 233 216]
  ...
  [251 255 234]
  [251 255 234]
  [251 255 235]]]
[[[250 252 252]
  [250 252 252]
  [250 252 252]
  ...
  [195 203 199]
  [202 210 203]
  [199 206 196]]

 [[250 252 252]
  [250 252 252]
  [250 252 252]
  ...
  [198 206 200]
  [201 208 200]
  [199 205 195]]

 [[249 253 252]
  [249 253 252]
  [249 253 251]
  ...
  [200 206 199]
  [200 205 196]
  [202 207 196]]

 ...

 [[255 237 209]
  [255 238 210]
  [255 238 210]
  ..

127.0.0.1 - - [27/Jul/2020 14:33:40] "GET / HTTP/1.1" 200 -


[[[248 253 252]
  [249 253 252]
  [249 253 251]
  ...
  [196 202 197]
  [203 210 203]
  [199 207 197]]

 [[248 253 252]
  [249 253 252]
  [249 253 251]
  ...
  [200 205 200]
  [203 209 201]
  [199 206 196]]

 [[248 253 252]
  [248 252 251]
  [248 252 250]
  ...
  [203 207 203]
  [202 207 199]
  [201 207 197]]

 ...

 [[255 236 211]
  [255 236 211]
  [255 237 212]
  ...
  [251 255 232]
  [251 255 232]
  [251 255 232]]

 [[255 236 211]
  [255 236 211]
  [255 236 211]
  ...
  [250 255 230]
  [250 255 230]
  [250 255 231]]

 [[255 235 210]
  [255 235 210]
  [255 234 209]
  ...
  [250 255 228]
  [250 255 229]
  [250 255 230]]]
[[[247 251 255]
  [247 251 255]
  [247 251 255]
  ...
  [206 208 200]
  [209 210 201]
  [209 209 201]]

 [[247 252 254]
  [247 252 254]
  [247 252 254]
  ...
  [207 209 200]
  [204 206 198]
  [205 205 198]]

 [[247 253 253]
  [247 253 253]
  [247 253 253]
  ...
  [205 207 199]
  [200 202 195]
  [203 205 199]]

 ...

 [[255 235 212]
  [255 236 213]
  [255 237 214]
  ..

In [6]:
vs.stop()

AttributeError: 'cv2.VideoCapture' object has no attribute 'stop'

[[[251 254 252]
  [250 253 251]
  [251 253 251]
  ...
  [196 207 200]
  [198 209 200]
  [200 211 201]]

 [[250 253 251]
  [250 253 251]
  [250 253 251]
  ...
  [199 209 202]
  [196 207 198]
  [199 210 200]]

 [[250 253 252]
  [250 253 252]
  [249 253 251]
  ...
  [200 208 201]
  [196 204 196]
  [202 211 201]]

 ...

 [[255 234 213]
  [255 234 213]
  [255 234 213]
  ...
  [251 255 233]
  [251 255 233]
  [250 255 232]]

 [[255 234 212]
  [255 234 212]
  [255 234 212]
  ...
  [251 255 232]
  [251 255 232]
  [250 255 232]]

 [[255 234 212]
  [255 234 212]
  [255 234 212]
  ...
  [251 255 231]
  [251 255 232]
  [250 255 231]]]
[[[252 253 251]
  [252 253 251]
  [251 252 250]
  ...
  [195 198 199]
  [203 205 206]
  [202 204 204]]

 [[251 253 251]
  [251 253 251]
  [251 252 250]
  ...
  [199 202 203]
  [204 206 206]
  [204 205 205]]

 [[249 253 251]
  [250 253 251]
  [250 252 250]
  ...
  [202 205 203]
  [204 206 204]
  [205 206 203]]

 ...

 [[255 235 213]
  [255 235 213]
  [255 236 212]
  ..

In [7]:
from werkzeug.wrappers import Request, Response
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Jul/2020 12:40:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jul/2020 12:40:41] "GET /favicon.ico HTTP/1.1" 404 -


TypeError: run() missing 1 required positional argument: 'self'

In [13]:
%tb

SystemExit: 1